In [1]:
from google.colab import drive
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm_notebook as tqdm
import numpy as np
from sklearn.metrics import r2_score,mean_squared_error
from math import sqrt
from sklearn.preprocessing import StandardScaler
import warnings;warnings.simplefilter('ignore')
import joblib
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# some function

In [0]:
def find_neighbors(one_sample):
  dist,ind = knn.kneighbors(tr.transform(one_sample[T_cols]))
  dist,ind = dist[0],ind[0]
  return train.iloc[ind]

def caculate_split_factor(neighbor):
  S1 = neighbor[npa_cols].values / neighbor['N+A'].values.reshape(-1,1)
  S2 = neighbor[np_ip_cols].values / np.array([ 100-i for i in neighbor['N+A'].values]).reshape(-1,1)
  # caculate neighbor's mean
  S1 = pd.DataFrame(S1.mean(axis=0)).T
  S1.columns = npa_cols
  S2 = pd.DataFrame(S2.mean(axis=0)).T
  S2.columns = np_ip_cols
  # normalize
  S1 = S1 / S1.sum(axis=1).values[0]
  S2 = S2 / S2.sum(axis=1).values[0]
  return S1,S2

def caculate_result(one_sample,S1,S2):
  a = S1*one_sample['N+A'].values[0] 
  b = S2*(100-one_sample['N+A'].values[0])
  return a.join(b)[y_cols]

def predict(X):
  neighbor = find_neighbors(X)
  S1,S2 = caculate_split_factor(neighbor)
  return caculate_result(X,S1,S2) 

# load raw data

In [3]:
train = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/phase1/train_4565.csv',index_col=0)
test = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/phase1/test_170.csv',index_col=0)
train.append(test).head()

,T10,T50,T90,N+A,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
2,100.5,119.2,146.5,31.978,0.272,0.132,0.059,5.293,2.570,2.819,0.494,10.395,8.070,6.411,2.917,9.138,9.649,4.810,5.373,6.405,9.759,4.590,3.661,0.875,5.257,0.525,0.319
3,99.6,117.9,145.5,31.568,0.297,0.151,0.062,5.089,2.531,2.804,0.499,10.074,7.958,6.395,2.894,8.970,9.548,4.753,5.443,6.324,9.899,4.301,2.995,0.881,5.591,1.119,0.303
4,100.0,118.8,145.6,31.344,0.262,0.126,0.061,5.107,2.571,2.771,0.492,10.069,7.913,6.378,2.890,9.006,9.591,4.778,5.468,6.360,9.983,4.274,2.979,0.865,5.641,0.964,0.289
5,100.4,118.6,142.9,31.453,0.224,0.105,0.060,4.894,2.497,2.650,0.469,10.015,7.685,6.376,2.866,9.133,9.708,4.889,5.510,6.444,10.182,4.420,2.964,0.830,5.637,0.968,0.281
6,100.4,118.1,142.2,32.190,0.243,0.117,0.061,4.946,2.503,2.695,0.477,10.053,7.765,6.394,2.877,9.101,9.676,4.855,5.500,6.416,10.115,4.347,3.725,0.835,4.823,0.969,0.290


# columns

In [0]:
cols = train.columns.tolist()

T_cols = cols[:3]
x_cols = cols[:4]
y_cols = cols[4:]

npa_cols = ['C5N',
            'C6N','C6A',
            'C7N','C7A',
            'C8N','C8A',
            'C9N','C9A',
            'C10N','C10A']

np_ip_cols = ['C5NP','C5IP',
              'C6NP','C6IP',
              'C7NP','C7IP',
              'C8NP','C8IP',
              'C9NP','C9IP',
              'C10NP','C10IP']

# define scaler and knn

In [0]:
# fit scaler on train T_cols
tr = StandardScaler().fit(train[T_cols])
# fit knn on train scaled_T_cols
knn = NearestNeighbors(n_neighbors=100).fit(tr.transform(train[T_cols]))

# predict

In [0]:
y_real = test[y_cols]
y_pred = test[y_cols].copy()

In [7]:
for idx in tqdm(test.index):
  X = test.loc[[idx],x_cols]
  y_pred.loc[idx,y_cols] = predict(X).values

In [8]:
res = pd.DataFrame(index=y_cols,columns=['R2','RMSE'])
for i in tqdm(y_cols):
  res.loc[i,'R2'] = r2_score(y_real[i],y_pred[i])
  res.loc[i,'RMSE'] = sqrt(mean_squared_error(y_real[i],y_pred[i]))
res.loc['AVG'] = res.mean(axis=0)
res

,R2,RMSE
C5NP,0.811195,0.371332
C5IP,0.605535,0.321396
C5N,0.868447,0.049653
C6NP,0.163824,0.369956
C6IP,0.869148,0.378107
C6N,0.869495,0.598251
C6A,-0.771073,0.227369
C7NP,0.899688,0.696555
C7IP,0.614974,0.685661
C7N,0.968853,0.735477


In [9]:
res = pd.DataFrame(columns=['y_pred','y_real'])
res['y_real'] = y_real[npa_cols].sum(axis=1).values
res['y_pred'] = y_pred[npa_cols].sum(axis=1).values
res

,y_pred,y_real
0,40.133,40.133
1,32.175,32.175
2,31.428,31.428
3,31.390,31.390
4,32.418,32.418
...,...,...
165,53.841,53.841
166,53.442,53.442
167,52.549,52.549
168,49.827,49.827


# compare

In [10]:
y_real.apply(lambda x:round(x,2)).head().style.background_gradient(axis=1)

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.14,0.62,0.22,3.74,2.58,4.03,0.56,7.67,5.99,10.21,3.08,7.92,8.04,7.04,5.44,5.26,9.54,4.88,3.48,1.09,5.65,0.62,0.6
1,0.12,0.08,0.03,3.96,1.56,2.41,0.4,10.02,7.26,6.57,2.81,9.54,9.82,5.14,5.48,6.79,10.85,4.31,3.96,1.07,6.02,0.58,0.48
2,0.1,0.06,0.02,3.92,1.45,2.3,0.4,10.24,7.35,6.26,2.8,9.65,9.99,4.94,5.45,6.95,11,4.27,4,1.03,6.11,0.54,0.46
3,0.12,0.08,0.02,3.9,1.44,2.27,0.4,10.26,7.36,6.23,2.82,9.67,10.15,4.93,5.48,6.95,11.03,4.26,3.98,1.01,6.05,0.55,0.45
4,0.07,0.05,0.02,3.87,1.37,2.21,0.49,10.21,7.32,6.01,2.97,9.56,10.05,4.77,6.58,6.87,10.89,4.17,4.23,0.93,5.83,0.52,0.46


In [11]:
y_pred.apply(lambda x:round(x,2)).head().style.background_gradient(axis=1)

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,0.98,0.66,0.13,3.71,2.21,3.43,0.93,8.33,6.62,9.19,3.41,8.21,8.38,6.8,6.09,5.27,9.2,5.12,3.63,1.2,5.11,0.71,0.68
1,0.25,0.18,0.03,3.13,1.12,2.19,0.47,10.53,7.95,6.54,2.89,9.71,10.29,5.13,5.61,6.57,10.94,4.31,3.56,1.18,5.98,0.67,0.78
2,0.28,0.2,0.03,2.86,1.04,2.07,0.4,10.57,8.12,6.42,2.77,9.74,10.35,5.02,5.31,6.71,11.26,4.36,3.47,1.25,6.21,0.67,0.9
3,0.19,0.13,0.03,3.16,1.1,2.1,0.46,10.66,8.04,6.35,2.86,9.9,10.44,5.02,5.52,6.71,11.17,4.21,3.51,1.13,5.97,0.61,0.73
4,0.25,0.18,0.03,3.09,1.11,2.18,0.47,10.44,7.89,6.55,2.91,9.65,10.25,5.13,5.66,6.57,10.96,4.35,3.65,1.18,6.02,0.67,0.82
